<a href="https://colab.research.google.com/github/mshojaei77/Awesome-Fine-tuning/blob/main/gemma_2_9b_qlora_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes datasets

In [19]:
from huggingface_hub import notebook_login
notebook_login()
from datasets import load_dataset

In [20]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma2 patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.7 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [22]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mshojaei77/merged_persian_alpaca", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/527456 [00:00<?, ? examples/s]

In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/527456 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 527,456 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 27,009,024


Step,Training Loss
1,2.973200
2,2.504100
3,2.082600
4,2.133000
5,2.281200
6,2.057700
7,1.953100
8,1.964600
9,2.058000
10,2.043300


In [30]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
input = 'سلام'
inputs = tokenizer([input], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
decode_outputs = tokenizer.batch_decode(outputs)
for decode_output in decode_outputs:
    print(decode_output)

<bos>سلام
من یک سال و نیم پیش یک وام از بانک گرفتم و الان میخوام وام جدید بگیرم ولی بانک قبول نمیکنه چون هنوز وام قبلی رو پرداخت نکردم.
میخواستم بدونم آیا میشه وام جدید رو از بانک دیگه ای بگیرم؟
با تشکر

سلام


In [25]:
model.push_to_hub("mshojaei77/persian_gemma-2-9b-4bit")
tokenizer.push_to_hub("mshojaei77/persian_gemma-2-9b-4bit")

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/108M [00:00<?, ?B/s]

Saved model to https://huggingface.co/mshojaei77/persian_gemma-2-9b-4bit


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [31]:
model.push_to_hub_merged("mshojaei77/persian_gemma-2-9b-4bit", tokenizer, save_method = "merged_16bit")

Unsloth: You are pushing to hub, but you passed your HF username = mshojaei77.
We shall truncate mshojaei77/persian_gemma-2-9b-4bit to persian_gemma-2-9b-4bit
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.93 out of 12.67 RAM for saving.


100%|██████████| 42/42 [08:15<00:00, 11.79s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving persian_gemma-2-9b-4bit/pytorch_model-00001-of-00004.bin...
Unsloth: Saving persian_gemma-2-9b-4bit/pytorch_model-00002-of-00004.bin...
Unsloth: Saving persian_gemma-2-9b-4bit/pytorch_model-00003-of-00004.bin...
Unsloth: Saving persian_gemma-2-9b-4bit/pytorch_model-00004-of-00004.bin...


README.md:   0%|          | 0.00/292 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/mshojaei77/persian_gemma-2-9b-4bit
